In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [3]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
ds = load_dataset("stanfordnlp/snli")
train_org = ds['train']
test_org = ds['test']
val_org = ds['validation']

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [5]:
train = train_org.select(range(0, len(train_org), 550))[:1000]
test = test_org.select(range(0, len(test_org), 100))[:100]
val = val_org.select(range(0, len(val_org), 100))[:100]

In [6]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type='nf4',bnb_4bit_compute_dtype=compute_dtype,bnb_4bit_use_double_quant=False)

In [63]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [64]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [65]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

import warnings
warnings.filterwarnings("ignore")

def create_prompt(premise, hypothesis):
    prompt = f"""Instruct: Check whether the hypothesis entails the premise or not. Follow the below format:
    0 -> The hypothesis entails the premise
    1 -> The premise and hypothesis neither entail nor contradict each other
    2 -> The hypothesis contradicts the premise
    Output only one single numerical value out of 0,1,2.

    Premise: {premise}
    Hypothesis: {hypothesis}
    
    Output:
    """
    return prompt
def get_answer(output):
    answer = None
    try:
        answer = int((output.split("Output:\n")[-1].strip().split("\n")[0]).split(" ")[0])
    except:
        answer = "Incorrect format"
    return answer

def zero_shot_inference(model, tokenizer, premise, hypothesis):
    prompt = create_prompt(premise, hypothesis)

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda" if torch.cuda.is_available() else "cpu")
    model.to(input_ids.device)

    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=input_ids.size(1)+64)

    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    answer = get_answer(output)
    return answer

pretrained_labels = []

correct_responses = 0
total = len(test['premise'])
incorrect_format = 0
for index in range(0, len(test['premise'])):
    if(index%11 == 0):
        print(index)

    premise = test['premise'][index]
    hypothesis = test['hypothesis'][index]
    original_label = test['label'][index]

    predicted_label = zero_shot_inference(original_model, tokenizer, premise, hypothesis)    
    if(predicted_label=="Incorrect format"):
        incorrect_format+=1
    if(int(predicted_label) == original_label):
        correct_responses+=1

    pretrained_labels.append(int(predicted_label))

accuracy = correct_responses*100/(total-incorrect_format)
print("Accuracy before fine-tuning: ", accuracy)
print("Samples with incorrect format: ", incorrect_format)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

77


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


88


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


99
Accuracy before fine-tuning:  54.0
Samples with incorrect format:  0
CPU times: user 6min, sys: 247 ms, total: 6min 1s
Wall time: 6min


In [28]:
from datasets import Dataset

In [29]:
dataset_test = Dataset.from_dict(test)
dataset_train = Dataset.from_dict(train)
dataset_val = Dataset.from_dict(val)

In [33]:
print(dataset_train)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 1000
})


In [37]:
def create_prompt_formats(sample):   
    premise = sample['premise']
    hypothesis = sample['hypothesis']
    original_label = sample['label']
    
    INTRO = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    
    prompt = f"""### Instruct: Check whether the hypothesis entails the premise or not. Follow the below format:
    0 -> The hypothesis entails the premise
    1 -> The premise and hypothesis neither entail nor contradict each other
    2 -> The hypothesis contradicts the premise
    Output only one single numerical value out of 0,1,2.

    Premise: {premise}
    Hypothesis: {hypothesis}
    
    ### Output: {original_label}
    """
    
    intro = f"\n{INTRO}"
    instruction = f"{prompt}"
    end = f"### End"
    formatted_prompt = intro+"\n\n"+ instruction+"\n\n"+end
    sample["text"] = formatted_prompt
    return sample

In [40]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

def preprocess_dataset(tokenizer, max_length,seed, dataset):
    
    print("Preprocessing")
    
    dataset = dataset.map(create_prompt_formats)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['premise', 'hypothesis', 'label'],
    )

    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    dataset = dataset.shuffle(seed=seed)
    return dataset

In [41]:
def get_datasets(train,val):
    train_dataset = preprocess_dataset(tokenizer, 2048,seed, dataset_train)
    val_dataset = preprocess_dataset(tokenizer, 2048,seed, dataset_val)
    return train_dataset,val_dataset

In [42]:
from functools import partial

max_length = get_max_length(original_model)
print("Max Length: ", max_length)
train_dataset,val_dataset = get_datasets(dataset_train,dataset_val)

Found max lenth: 2048
Max Length:  2048
Preprocessing


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Preprocessing


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [52]:
val_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [43]:
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [44]:
original_model = prepare_model_for_kbit_training(original_model)

In [45]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [47]:
def print_number_of_trainable_model_parameters(model):

    trainable_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
    total_params = sum(param.numel() for param in model.parameters())
    trainable_percentage = (trainable_params / total_params) * 100 if total_params > 0 else 0
    print(f"Trainable Parameters: {trainable_params}")
    print(f"Total Parameters: {total_params}")
    print(f"Percentage of Trainable Parameters: {trainable_percentage:.2f}%")
    return trainable_params


In [48]:
print(print_number_of_trainable_model_parameters(peft_model))

Trainable Parameters: 20971520
Total Parameters: 1542364160
Percentage of Trainable Parameters: 1.36%
20971520


In [53]:
import time
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

output_dir = f'./checkpoints'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    warmup_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-4,
    optim="adamw_8bit",
    logging_steps=100,
    logging_dir="./logs",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    do_eval=True,
    gradient_checkpointing=False,
    report_to="none",
    overwrite_output_dir=False,
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

start_time = time.time()  # Start time

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()  # Synchronize before starting training

# Training
peft_trainer.train()

end_time = time.time()
training_time = end_time - start_time

if torch.cuda.is_available():
    peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert to GB
    print(f"Peak Memory Usage (GB): {peak_memory:.2f}")
else:
    peak_memory = None
    print("CUDA not available; unable to track peak memory usage.")

print(f"Time taken to fine-tune the model (seconds): {training_time:.2f}")

Epoch,Training Loss,Validation Loss
1,0.598300,0.492091
2,0.457100,0.481063
3,0.429400,0.478328
4,0.392600,0.482428
5,0.381200,0.491142


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Peak Memory Usage (GB): 3.24
Time taken to fine-tune the model (seconds): 2270.90


In [54]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [55]:
import os

lst = os.listdir("/kaggle/working/checkpoints")

for file in lst:
    print(f"/kaggle/working/checkpoints/{file}")

/kaggle/working/checkpoints/checkpoint-250
/kaggle/working/checkpoints/checkpoint-625
/kaggle/working/checkpoints/checkpoint-125
/kaggle/working/checkpoints/checkpoint-500
/kaggle/working/checkpoints/checkpoint-375


In [56]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/checkpoints/checkpoint-625",torch_dtype=torch.float16,is_trainable=False)

In [66]:
correct_responses = 0
total = len(test['premise'])
incorrect_format = 0

finetuned_labels = []

for index in range(0, len(test['premise'])):
    if(index%11 == 0):
        print(index)

    premise = test['premise'][index]
    hypothesis = test['hypothesis'][index]
    original_label = test['label'][index]

    predicted_label = zero_shot_inference(ft_model, tokenizer, premise, hypothesis)    
    if(predicted_label=="Incorrect format"):
        incorrect_format+=1
    if(int(predicted_label) == original_label):
        correct_responses+=1

    finetuned_labels.append(int(predicted_label))

accuracy = correct_responses*100/(total-incorrect_format)
print("Accuracy after fine-tuning: ", accuracy)
print("Samples with incorrect format: ", incorrect_format)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

77


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


88


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


99
Accuracy after fine-tuning:  87.0
Samples with incorrect format:  0


In [59]:
### Downloading the Model Checkpoints
!zip -r file.zip /kaggle/working/checkpoints

!ls

from IPython.display import FileLink
FileLink(r'file.zip')

import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))
                
    file_size = os.path.getsize(file_name)
    readable_size = file_size / (1024 ** 2)  # Convert to MB
    print(f"Created zip file '{file_name}' with size: {readable_size:.2f} MB")

    return FileLink(file_name)
zip_dir()

  adding: kaggle/working/checkpoints/ (stored 0%)
  adding: kaggle/working/checkpoints/checkpoint-250/ (stored 0%)
  adding: kaggle/working/checkpoints/checkpoint-250/scheduler.pt (deflated 56%)
  adding: kaggle/working/checkpoints/checkpoint-250/optimizer.pt (deflated 10%)
  adding: kaggle/working/checkpoints/checkpoint-250/rng_state.pth (deflated 25%)
  adding: kaggle/working/checkpoints/checkpoint-250/README.md (deflated 66%)
  adding: kaggle/working/checkpoints/checkpoint-250/adapter_config.json (deflated 53%)
  adding: kaggle/working/checkpoints/checkpoint-250/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/checkpoints/checkpoint-250/trainer_state.json (deflated 62%)
  adding: kaggle/working/checkpoints/checkpoint-250/training_args.bin (deflated 52%)
  adding: kaggle/working/checkpoints/checkpoint-625/ (stored 0%)
  adding: kaggle/working/checkpoints/checkpoint-625/scheduler.pt (deflated 56%)
  adding: kaggle/working/checkpoints/checkpoint-625/optimizer.pt (deflat

/kaggle/working/directory.zip

In [74]:
### Comparison

### Printing 5 cases where pretrained labels were incorrect and finetuned labels were correct
count=0
ct=0
for i in range(len(test['label'])):
    original_label = test['label'][i]
    pretrained_label = pretrained_labels[i]
    finetuned_label = finetuned_labels[i]

    if(pretrained_label != original_label and finetuned_label == original_label):
        if(ct<5):
            print("-----------"+str(ct+1)+"-----------")
            print("Premise: ", test['premise'][i])
            print("Hypothesis: ", test['hypothesis'][i])
            print("Label: ", test['label'][i])
            print("Pretrained Label: ", pretrained_labels[i])
            print("Finetuned Label: ", finetuned_labels[i])
        ct+=1
        count+=1

print("Number of cases that were corrected by the fine-tuned model:",count)

-----------1-----------
Premise:  a woman in a black shirt looking at a bicycle.
Hypothesis:  A woman dressed in black shops for a bicycle.
Label:  1
Pretrained Label:  0
Finetuned Label:  1
-----------2-----------
Premise:  A group of people stand near and on a large black square on the ground with some yellow writing on it.
Hypothesis:  a group of people wait
Label:  1
Pretrained Label:  0
Finetuned Label:  1
-----------3-----------
Premise:  Two men in neon yellow shirts busily sawing a log in half.
Hypothesis:  Two men are cutting wood to build a table.
Label:  1
Pretrained Label:  0
Finetuned Label:  1
-----------4-----------
Premise:  A man is renovating a room.
Hypothesis:  A man is using a hammer in a room.
Label:  1
Pretrained Label:  0
Finetuned Label:  1
-----------5-----------
Premise:  An Ambulance is passing a man wearing a bandanna and girl.
Hypothesis:  The man in the bandana is running after the ambulance
Label:  2
Pretrained Label:  1
Finetuned Label:  2
Number of cas

In [75]:
### Printing 5 cases where pretrained labels were incorrect and finetuned labels were also incorrect
count=0
ct=0
for i in range(len(test['label'])):
    original_label = test['label'][i]
    pretrained_label = pretrained_labels[i]
    finetuned_label = finetuned_labels[i]

    if(pretrained_label != original_label and finetuned_label != original_label):
        if(ct<5):
            print("-----------"+str(ct+1)+"-----------")
            print("Premise: ", test['premise'][i])
            print("Hypothesis: ", test['hypothesis'][i])
            print("Label: ", test['label'][i])
            print("Pretrained Label: ", pretrained_labels[i])
            print("Finetuned Label: ", finetuned_labels[i])
        ct+=1
        count+=1

print("Number of cases that were not corrected by the fine-tuned model:",count)

-----------1-----------
Premise:  This church choir sings to the masses as they sing joyous songs from the book at a church.
Hypothesis:  The church has cracks in the ceiling.
Label:  1
Pretrained Label:  2
Finetuned Label:  2
-----------2-----------
Premise:  Two men climbing on a wooden scaffold.
Hypothesis:  Two sad men climbing on a wooden scaffold.
Label:  1
Pretrained Label:  2
Finetuned Label:  2
-----------3-----------
Premise:  A woman is standing near three stores, two have beautiful artwork and the other store has Largo written on it.
Hypothesis:  A woman standing on a street corner outside beside three different stores, two of which contain beautiful artwork and one with a Largo sign.
Label:  0
Pretrained Label:  1
Finetuned Label:  1
-----------4-----------
Premise:  Military personnel are shopping
Hypothesis:  Military personnel are in the mall.
Label:  1
Pretrained Label:  0
Finetuned Label:  0
-----------5-----------
Premise:  An older gentleman wearing a hat is walking